In [13]:
# %load_ext gradio

In [14]:
# autodl内置学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'

In [15]:

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append('/root/StickyToken')
import torch
import gradio as gr
import json
import pandas as pd
import random
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from stickytoken.utils import load_verification_results
from collections import namedtuple
DistanceMetrics = namedtuple("Metrics", ["cosine_distance", "euclidean_distance", "manhattan_distance"])
from stickytoken.utils import distance_metrics, random_insert
from stickytoken.sentence_pair import output_dataset_name

In [ ]:
path = "/root/StickyToken/results/experiment_information.json"
try:
    with open(path, 'r', encoding='utf-8') as f:
        experiement_record = json.load(f)
except (FileNotFoundError, json.JSONDecodeError):
    experiement_record = []
experiement_record_df = pd.DataFrame(experiement_record)
model_list = experiement_record_df["model_name"].tolist()
experiement_record_df.head(3)

In [ ]:
model_list

In [ ]:
model_names = [
    "all-MiniLM-L6-v2", "all-mpnet-base-v2",
    "sup-simcse-bert-base-uncased", "sup-simcse-bert-large-uncased", "sup-simcse-roberta-base", "sup-simcse-roberta-large",
    "sentence-t5-base", "sentence-t5-large", "sentence-t5-xl", "sentence-t5-xxl",
    "gtr-t5-base", "gtr-t5-large", "gtr-t5-xl", "gtr-t5-xxl",
    "instructor-base", "instructor-large", "instructor-xl",
    "e5-small", "e5-base", "e5-large", "e5-mistral-7b-instruct",
    "bge-small-en-v1.5", "bge-base-en-v1.5", "bge-large-en-v1.5",
    "UAE-Large-V1",
    "nomic-embed-text-v1", "nomic-embed-text-v1.5",
    "gte-small", "gte-base", "gte-large", "gte-base-en-v1.5", "gte-large-en-v1.5", "gte-Qwen2-1.5B-instruct", "gte-Qwen2-7B-instruct",
    "GritLM-7B",
    "SFR-Embedding-2_R", "SFR-Embedding-Mistral",
]
# 按照model_names的顺序对DataFrame进行排序
experiement_record_df = experiement_record_df.set_index('model_name').loc[model_names].reset_index()
experiement_record_df[['model_name', 'num_parameters', 'vocab_size']].assign(
    num_parameters=lambda x: (x['num_parameters'] / 1e6).round().astype(int).astype(str) + 'M'
)

In [18]:
# num = 250
# model_name = model_list[0]
# sampled_sentence_pairs_path = output_dataset_name(model_list[0], 'sampled_sentence_pairs', 'csv')
# dataset = load_dataset('csv', data_files=sampled_sentence_pairs_path, split='train')

# json_file = os.path.join('/root/StickyToken/magicembed', 'model_record.json')
# with open(json_file, 'r') as f:
#     records = json.load(f)
                
#     model_name = model_name  # 假设moda有一个属性model_name
#     mean_similarity = next((np.float32(record["vocab_embeddings_mean_cosine_similarity"]) for record in records if record['model_name'] == model_name), None)

# filtered_dataset = dataset.filter(lambda x: x['similarity'] < mean_similarity)
        
        
# min_similarity = min(filtered_dataset['similarity'])
# max_similarity = min_similarity + (mean_similarity - min_similarity) * 0.8
#         # 判断是否有足够的句子对
# if len(filtered_dataset) < num:
#         print(f"只有 {len(filtered_dataset)} 个句子对，无法采样 {num} 个。建议采样{len(filtered_dataset)/2}。可以适当增加SentencePair.get_sampled_sentence_pairs()的sample_size参数 > sample_size * {num/len(filtered_dataset)}")
#         num = len(filtered_dataset)

# # 计算相似度步长
# similarity_step = (max_similarity - min_similarity) / (num - 1)

# # 进行近似均匀采样
# sampled_indices = []
# for i in range(num):
#     target_similarity = min_similarity + i * similarity_step
#     closest_index = np.argsort(np.abs(filtered_dataset['similarity'] - target_similarity))[0]
#     sampled_indices.append(closest_index)

# sampled_dataset = filtered_dataset.select(sampled_indices)
# nested_list = [list(pair) for pair in zip(sampled_dataset['sentence1'], sampled_dataset['sentence2'])]

In [19]:

# verification_results = load_verification_results(model_list[0])
# verification_results_df = pd.DataFrame(verification_results).transpose()
# strong_verified_tokens = verification_results_df[verification_results_df["magic"] == "strong_verified"]
# magic_token = strong_verified_tokens["raw_vocab"]
# magic_token = strong_verified_tokens["raw_vocab"]
# magic_token_num = len(magic_token)

In [20]:
# model = SentenceTransformer('sentence-transformers/sentence-t5-base')

In [21]:

# def token_process_pipline(token: str,
#                             gt_texts: list[str]|str,
#                             contract_texts: list[str]|str,
#                             num: int,
#                             model,
#                             ):
#         if isinstance(gt_texts, str):
#             gt_texts = [gt_texts]
#         if isinstance(contract_texts, str):
#             contract_texts = [contract_texts]
#         gt_embs = model.encode(gt_texts)
#         contract_embs = model.encode(contract_texts)
#         gt_metrics = distance_metrics(gt_embs,contract_embs)

#         results = {'Prefix': [], 'Suffix': [], 'Insert': []}

#         def process_texts(text_list, gt_emb, gt_metric, text_type, id):
#             pred_emb = model.encode(text_list)
            
#             ret = distance_metrics(gt_emb, pred_emb)

#             result = {
#                 'Pair_id': id,  
#                 'Source text': gt_texts[id],
#                 'Texts to be contrasted': [contract_text] + text_list,
#                 'cosine_distance': [gt_metric.cosine_distance] + list(ret.cosine_distance),
#                 'cosine_distance_contrast': [None],
#                 'euclidean_distance': [gt_metric.euclidean_distance] + list(ret.euclidean_distance),
#                 'euclidean_distance_contrast': [None],
#                 'manhattan_distance': [gt_metric.manhattan_distance] + list(ret.manhattan_distance),
#                 'manhattan_distance_contrast': [None],
#             }

#             for i in range(num):
#                 cosine_distance_contrast = (result['cosine_distance'][i] > result['cosine_distance'][i + 1])
#                 result['cosine_distance_contrast'].append(cosine_distance_contrast)
#                 euclidean_distance_contrast = (result['euclidean_distance'][i] > result['euclidean_distance'][i + 1])
#                 result['euclidean_distance_contrast'].append(euclidean_distance_contrast)
#                 manhattan_distance_contrast = (result['manhattan_distance'][i] > result['manhattan_distance'][i + 1])
#                 result['manhattan_distance_contrast'].append(manhattan_distance_contrast)

#             results[text_type].append(result)

#         for id, contract_text in enumerate(contract_texts):
#             text_list_prefix = [token * i + contract_text for i in range(1, num + 1)]
#             text_list_suffix = [contract_text + token * i for i in range(1, num + 1)]
#             text_list_insert = []
#             tem_text = contract_text
#             for i in range(1, num + 1):
#                 new_text = random_insert(tem_text, token, 1)
#                 tem_text = new_text
#                 text_list_insert.append(new_text)
            
#             for text_list, text_type in [(text_list_prefix, 'Prefix'), (text_list_suffix, 'Suffix'), (text_list_insert, 'Insert')]:
#                 process_texts(text_list, gt_embs[id],
#                             DistanceMetrics(
#                                     gt_metrics.cosine_distance[id],
#                                     gt_metrics.euclidean_distance[id],
#                                     gt_metrics.manhattan_distance[id]
#                                 )
#                                 , text_type, id)
#         return pd.DataFrame({
#         'Prefix_text': results['Prefix'][0]['Texts to be contrasted'],
#         'Suffix_text': results['Suffix'][0]['Texts to be contrasted'], 
#         'Insert-text': results['Insert'][0]['Texts to be contrasted'],
#         'Prefix': results['Prefix'][0]['cosine_distance'],
#         'Suffix': results['Suffix'][0]['cosine_distance'], 
#         'Insert': results['Insert'][0]['cosine_distance'],
#         'num': range(len(results['Prefix'][0]['cosine_distance']))
#     })

In [22]:
# token_process_pipline('<s>', ["hello"], ["good morning"], 8, model)

In [23]:
# result = token_process_pipline('<s>', ["hello"], ["good morning"], 8, model)
# result['Prefix'][0]['cosine_distance']
# result_df = pd.DataFrame({
#     'Prefix': result['Prefix'][0]['cosine_distance'],
#     'Suffix': result['Suffix'][0]['cosine_distance'], 
#     'Insert': result['Insert'][0]['cosine_distance'],
#     'num': range(len(result['Prefix'][0]['cosine_distance']))
# })

In [24]:
def build_model_ui():

    def update_model_path(selected_model_name):
        model_path = experiement_record_df.loc[experiement_record_df["model_name"] == selected_model_name, "model"].values[0]
        return model_path
    
    model_path = update_model_path(model_list[0])

    def update_magic_token(selected_model_name):
        verification_results = load_verification_results(selected_model_name)
        verification_results_df = pd.DataFrame(verification_results).transpose()
        strong_verified_tokens = verification_results_df[verification_results_df["magic"] == "strong_verified"]
        token_list = strong_verified_tokens["raw_vocab"].tolist()
        
        if len(token_list)<6:
            weak_verified_tokens = verification_results_df[verification_results_df["magic"] == "weak_verified"]
            token_list = weak_verified_tokens["raw_vocab"].tolist()
        
        return token_list, len(token_list)
    
    def update_model(selected_model_name):
        torch.cuda.empty_cache()
        new_model_path = update_model_path(selected_model_name)
        model = SentenceTransformer(str(new_model_path))
        return model

    def update_examples(model_name):
        num = 250
        sampled_sentence_pairs_path = output_dataset_name(model_name, 'sampled_sentence_pairs', 'csv')
        dataset = load_dataset('csv', data_files=sampled_sentence_pairs_path, split='train')

        json_file = os.path.join('/root/StickyToken/magicembed', 'model_record.json')
        with open(json_file, 'r') as f:
            records = json.load(f)
            mean_similarity = next((np.float32(record["vocab_embeddings_mean_cosine_similarity"]) for record in records if record['model_name'] == model_name), None)

        filtered_dataset = dataset.filter(lambda x: x['similarity'] < mean_similarity)
                
        min_similarity = min(filtered_dataset['similarity'])
        max_similarity = min_similarity + (mean_similarity - min_similarity) * 0.8
                # 判断是否有足够的句子对
        if len(filtered_dataset) < num:
            num = len(filtered_dataset)

        # 计算相似度步长
        similarity_step = (max_similarity - min_similarity) / (num - 1)

        # 进行近似均匀采样
        sampled_indices = []
        for i in range(num):
            target_similarity = min_similarity + i * similarity_step
            closest_index = np.argsort(np.abs(filtered_dataset['similarity'] - target_similarity))[0]
            sampled_indices.append(closest_index)

        sampled_dataset = filtered_dataset.select(sampled_indices)
        nested_list = [list(pair) for pair in zip(sampled_dataset['sentence1'], sampled_dataset['sentence2'])]
        random.shuffle(nested_list)
        return gr.Dataset(samples=nested_list)

    def init_examples(model_name):
            num = 250
            sampled_sentence_pairs_path = output_dataset_name(model_name, 'sampled_sentence_pairs', 'csv')
            dataset = load_dataset('csv', data_files=sampled_sentence_pairs_path, split='train')

            json_file = os.path.join('/root/StickyToken/magicembed', 'model_record.json')
            with open(json_file, 'r') as f:
                records = json.load(f)
                mean_similarity = next((np.float32(record["vocab_embeddings_mean_cosine_similarity"]) for record in records if record['model_name'] == model_name), None)

            filtered_dataset = dataset.filter(lambda x: x['similarity'] < mean_similarity)
                    
            min_similarity = min(filtered_dataset['similarity'])
            max_similarity = min_similarity + (mean_similarity - min_similarity) * 0.8
                    # 判断是否有足够的句子对
            if len(filtered_dataset) < num:
                num = len(filtered_dataset)

            # 计算相似度步长
            similarity_step = (max_similarity - min_similarity) / (num - 1)

            # 进行近似均匀采样
            sampled_indices = []
            for i in range(num):
                target_similarity = min_similarity + i * similarity_step
                closest_index = np.argsort(np.abs(filtered_dataset['similarity'] - target_similarity))[0]
                sampled_indices.append(closest_index)

            sampled_dataset = filtered_dataset.select(sampled_indices)
            nested_list = [list(pair) for pair in zip(sampled_dataset['sentence1'], sampled_dataset['sentence2'])]
            random.shuffle(nested_list)
            return nested_list

    model = update_model(model_list[0])

    def token_process_pipline(token: str,
                              gt_texts: list[str] | str,
                              contract_texts: list[str] | str,
                              num: int,
                              model=model):
        if isinstance(gt_texts, str):
            gt_texts = [gt_texts]
        if isinstance(contract_texts, str):
            contract_texts = [contract_texts]
        gt_embs = model.encode(gt_texts)
        contract_embs = model.encode(contract_texts)
        gt_metrics = distance_metrics(gt_embs, contract_embs)

        results = {'Prefix': [], 'Suffix': [], 'Insert': []}

        def process_texts(text_list, gt_emb, gt_metric, text_type, id):
            pred_emb = model.encode(text_list)
            ret = distance_metrics(gt_emb, pred_emb)

            result = {
                'Pair_id': id,  
                'Source text': gt_texts[id],
                'Texts to be contrasted': [contract_text] + text_list,
                'cosine_distance': [gt_metric.cosine_distance] + list(ret.cosine_distance),
                'cosine_distance_contrast': [None],
                'euclidean_distance': [gt_metric.euclidean_distance] + list(ret.euclidean_distance),
                'euclidean_distance_contrast': [None],
                'manhattan_distance': [gt_metric.manhattan_distance] + list(ret.manhattan_distance),
                'manhattan_distance_contrast': [None],
            }

            for i in range(num):
                cosine_distance_contrast = (result['cosine_distance'][i] > result['cosine_distance'][i + 1])
                result['cosine_distance_contrast'].append(cosine_distance_contrast)
                euclidean_distance_contrast = (result['euclidean_distance'][i] > result['euclidean_distance'][i + 1])
                result['euclidean_distance_contrast'].append(euclidean_distance_contrast)
                manhattan_distance_contrast = (result['manhattan_distance'][i] > result['manhattan_distance'][i + 1])
                result['manhattan_distance_contrast'].append(manhattan_distance_contrast)

            results[text_type].append(result)

        for id, contract_text in enumerate(contract_texts):
            text_list_prefix = [token * i + contract_text for i in range(1, num + 1)]
            text_list_suffix = [contract_text + token * i for i in range(1, num + 1)]
            text_list_insert = []
            tem_text = contract_text
            for i in range(1, num + 1):
                new_text = random_insert(tem_text, token, 1)
                tem_text = new_text
                text_list_insert.append(new_text)
            
            for text_list, text_type in [(text_list_prefix, 'Prefix'), (text_list_suffix, 'Suffix'), (text_list_insert, 'Insert')]:
                process_texts(text_list, gt_embs[id],
                              DistanceMetrics(
                                  gt_metrics.cosine_distance[id],
                                  gt_metrics.euclidean_distance[id],
                                  gt_metrics.manhattan_distance[id]
                              ), text_type, id)
        return pd.DataFrame({
            'num': range(len(results['Prefix'][0]['cosine_distance'])),
            'Prefix_text': results['Prefix'][0]['Texts to be contrasted'],
            'Prefix': [round(x, 2) for x in results['Prefix'][0]['cosine_distance']],
            'Suffix_text': results['Suffix'][0]['Texts to be contrasted'], 
            'Suffix': [round(x, 2) for x in results['Suffix'][0]['cosine_distance']],
            'Insert-text': results['Insert'][0]['Texts to be contrasted'],
            'Insert': [round(x, 2) for x in results['Insert'][0]['cosine_distance']]
        }).round(2)
    
    with gr.Column():
        model_selector = gr.Dropdown(
            choices=model_list,
            label="Model",
            value=model_list[0],
            interactive=True,
            show_label=False
        )
        with gr.Row():
            model_size = gr.Textbox(experiement_record_df.loc[experiement_record_df["model_name"] == model_list[0], "num_parameters"].values[0], label="Model Size")
            magic_token_static = gr.Textbox(update_magic_token(model_list[0])[1], label="Magic Token Number")
            vocab_size = gr.Textbox(experiement_record_df.loc[experiement_record_df["model_name"] == model_list[0], "vocab_size"].values[0], label="Vocab Size")
        magic_token = gr.Textbox(update_magic_token(model_list[0])[0], label="Magic Token",
                                 max_lines=7,)


    # with gr.Row():
    #     with gr.Accordion(
    #         "🔍 Expand to see all model descriptions",
    #         open=False,
    #         elem_id="model_description_accordion",
    #     ):
            # model_description_md = models.get_model_description_md(task_type="sts")
            # gr.Markdown(model_description_md, elem_id="model_description_markdown")
            # gr.Markdown("test")
    
    empty_df = pd.DataFrame({'num': [], 
                             'Prefix_text': [],
                             'Prefix': [], 
                             'Suffix_text': [],
                             'Suffix': [], 
                             'Insert-text': [],
                             'Insert': [],
                            })
    result_df = gr.DataFrame(empty_df)

    with gr.Row():
        with gr.Column():
            token = gr.Textbox(
                value=update_magic_token(model_list[0])[0][0],
                label="Token",
                placeholder="Please input magic token",
            )
            insrt_num = gr.Slider(value=8, minimum=1, maximum=20, step=1, label="Insert Number")
            insert_method = gr.CheckboxGroup(['Prefix', 'Suffix', 'Insert'], label="insert_method", value=['Prefix', 'Suffix', 'Insert'])

            gt_text = gr.Textbox(
                value="hello",
                label="Ground Truth Text",
                placeholder="Please input ground truth",
            )
            contract_texts = gr.Textbox(
                value="good morning",
                label="Contract Texts",
                placeholder="Please input contract texts",
            )
            with gr.Row():
                draw_btn = gr.Button(value="🎲 Random sentences", variant="primary", scale=1)
                draw_token_btn = gr.Button(value="🎲 Random token", variant="primary", scale=1)
            send_btn = gr.Button(value="Send", variant="primary", scale=1)
        with gr.Column():
            plt_prefix = gr.LinePlot(empty_df, x="num", y="Prefix")
            plt_suffix = gr.LinePlot(empty_df, x="num", y="Suffix")
            plt_insert = gr.LinePlot(empty_df, x="num", y="Insert")
            plots = [plt_prefix, plt_suffix, plt_insert]

            def select_region(selection: gr.SelectData):
                min_w, max_w = selection.index
                return [gr.LinePlot(x_lim=(min_w, max_w))] * len(plots) 

            for plt in plots:
                plt.select(select_region, None, plots)
                plt.double_click(lambda: [gr.LinePlot(x_lim=None)] * len(plots), None, plots)

    example = gr.Examples(
        examples=init_examples(model_list[0]),
        inputs=[gt_text, contract_texts],
    )

    model_selector.change(
        fn=update_model_path, inputs=model_selector, outputs=None
    ).then(
        fn=lambda x: experiement_record_df.loc[experiement_record_df["model_name"] == x, "num_parameters"].values[0],
        inputs=model_selector,
        outputs=model_size
    ).then(
        fn=lambda x: experiement_record_df.loc[experiement_record_df["model_name"] == x, "vocab_size"].values[0],
        inputs=model_selector,
        outputs=vocab_size
    ).then(
        fn=update_magic_token, inputs=model_selector,
        outputs=[magic_token, magic_token_static]
    ).then(
        fn=update_examples, inputs=model_selector,
        outputs=example.dataset
    ).success(
        fn=update_model, inputs=model_selector, outputs=None
    )

    draw_btn.click(
        fn=lambda: random.choice(example.examples),
        inputs=None,
        outputs=[gt_text, contract_texts]
    )

    draw_token_btn.click(
        fn=lambda x: random.choice(update_magic_token(x)[0]),
        inputs=model_selector,
        outputs=token
    )

    send_btn.click(
        fn=token_process_pipline,
        inputs=[token, gt_text, contract_texts, insrt_num],
        outputs=result_df,
    ).then(
        fn=lambda df: [gr.LinePlot(df, x="num", y=col) for col in ["Prefix", "Suffix", "Insert"]],
        inputs=result_df,
        outputs=plots
    )


In [ ]:
with gr.Blocks(title="Magic Token") as block:
    notice_markdown = f"""
# 💧 Magic Token ☘️
"""
    gr.Markdown(notice_markdown, elem_id="notice_markdown")

    gr.Markdown("Adding these tokens into sentence pairs results in atypical changes to the similarity of their text embeddings. We conduct an exhaustive analysis of the tokenizer mechanisms in text embedding models, focusing on identifying these “magic tokens”. By leveraging tokenizer examination, indicators, and validation procedures, we develop effective methods for automatically detecting these problematic tokens.")

    with gr.Tab("☘️ Model"):
        build_model_ui()
    
    with gr.Tab("🔍 Table"):
        # 展示experiement_record_df,去除不需要的列
        gr.DataFrame(experiement_record_df.drop(columns=["record_time", "model","dataset","insert_num","sent_pair_num","verification_sent_pair_num","vocab_embeddings_is_on_unit_sphere",
        "wte_is_on_unit_sphere",
        "vocab_embeddings_is_anisotropic",
        "wte_is_anisotropic",]).round(2))

block.launch()